In [1]:
# %%
import re, json, time
from selenium import webdriver
import os

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


In [2]:
def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector


def scrape_all_data(selector):
    youtube_video_page = []

    all_script_tags = selector.css("script").getall()

    title = selector.css(".title .ytd-video-primary-info-renderer::text").get()

    # https://regex101.com/r/gHeLwZ/1
    views = int(
        re.search(r"(.*)\s", selector.css(".view-count::text").get())
        .group()
        .replace(",", "")
    )

    # https://regex101.com/r/9OGwJp/1
    likes = int(
        re.search(
            r"(.*)\s",
            selector.css(
                "#top-level-buttons-computed > ytd-toggle-button-renderer:first-child #text::attr(aria-label)"
            ).get(),
        )
        .group()
        .replace(",", "")
    )

    date = selector.css("#info-strings yt-formatted-string::text").get()

    duration = selector.css(".ytp-time-duration::text").get()

    # https://regex101.com/r/0JNma3/1
    keywords = (
        "".join(
            re.findall(r'"keywords":\[(.*)\],"channelId":".*"', str(all_script_tags))
        )
        .replace('"', "")
        .split(",")
    )

    # https://regex101.com/r/9VhH1s/1
    thumbnail = re.findall(
        r'\[{"url":"(\S+)","width":\d*,"height":\d*},', str(all_script_tags)
    )[0].split('",')[0]

    channel = {
        # https://regex101.com/r/xFUzq5/1
        "id": "".join(
            re.findall(r'"channelId":"(.*)","isOwnerViewing"', str(all_script_tags))
        ),
        "name": selector.css("#channel-name a::text").get(),
        "link": f'https://www.youtube.com{selector.css("#channel-name a::attr(href)").get()}',
        "subscribers": selector.css("#owner-sub-count::text").get(),
        "thumbnail": selector.css("#img::attr(src)").get(),
    }

    description = selector.css(
        ".ytd-expandable-video-description-body-renderer span:nth-child(1)::text"
    ).get()

    hash_tags = [
        {
            "name": hash_tag.css("::text").get(),
            "link": f'https://www.youtube.com{hash_tag.css("::attr(href)").get()}',
        }
        for hash_tag in selector.css(
            ".ytd-expandable-video-description-body-renderer a"
        )
    ]

    # https://regex101.com/r/onRk9j/1
    category = "".join(
        re.findall(r'"category":"(.*)","publishDate"', str(all_script_tags))
    )

    comments_amount = int(
        selector.css("#count .count-text span:nth-child(1)::text")
        .get()
        .replace(",", "")
    )

    comments = []

    for comment in selector.css("#contents > ytd-comment-thread-renderer"):
        comments.append(
            {
                "author": comment.css("#author-text span::text").get().strip(),
                "link": f'https://www.youtube.com{comment.css("#author-text::attr(href)").get()}',
                "date": comment.css(".published-time-text a::text").get(),
                "likes": comment.css("#vote-count-middle::text").get().strip(),
                "comment": comment.css("#content-text::text").get(),
                "avatar": comment.css("#author-thumbnail #img::attr(src)").get(),
            }
        )

    suggested_videos = []

    for video in selector.css("ytd-compact-video-renderer"):
        suggested_videos.append(
            {
                "title": video.css("#video-title::text").get().strip(),
                "link": f'https://www.youtube.com{video.css("#thumbnail::attr(href)").get()}',
                "channel_name": video.css("#channel-name #text::text").get(),
                "date": video.css("#metadata-line span:nth-child(2)::text").get(),
                "views": video.css("#metadata-line span:nth-child(1)::text").get(),
                "duration": video.css("#overlays #text::text").get().strip(),
                "thumbnail": video.css("#thumbnail img::attr(src)").get(),
            }
        )

    youtube_video_page.append(
        {
            "title": title,
            "views": views,
            "likes": likes,
            "date": date,
            "duration": duration,
            "channel": channel,
            "keywords": keywords,
            "thumbnail": thumbnail,
            "description": description,
            "hash_tags": hash_tags,
            "category": category,
            "suggested_videos": suggested_videos,
            "comments_amount": comments_amount,
            "comments": comments,
        }
    )

    print(json.dumps(youtube_video_page, indent=2, ensure_ascii=False))


def get_info_from_channel(channel_link: str):
    driver = webdriver.Chrome()

    driver.get(channel_link)
    driver.maximize_window()

    time.sleep(5)

    selector = Selector(text=driver.page_source)
    driver.quit()

    # channel title
    channel_title = selector.css("#text-container #text::text").get()

    # date joined //*[@id="right-column"]/yt-formatted-string[2]/span[2]
    date_joined = selector.css(
        "#right-column > yt-formatted-string:nth-child(2) > span:nth-child(2)::text"
    ).get()

    # total views //*[@id="right-column"]/yt-formatted-string[3]

    total_views = selector.css(
        "#right-column > yt-formatted-string:nth-child(3)::text"
    ).get()

    # total subs //*[@id="subscriber-count"]

    total_subs = selector.css("#subscriber-count::text").get()

    # total videos //*[@id="videos-count"]/span[1]

    total_videos = selector.css("#videos-count > span:nth-child(1)::text").get()

    channel_info = {
        "channel_title": channel_title,
        "date_joined": date_joined,
        "total_views": total_views,
        "total_subs": total_subs,
        "total_videos": total_videos,
    }

    print(json.dumps(channel_info, indent=2, ensure_ascii=False))

    return pd.DataFrame(channel_info, index=[0])


def get_info_from_video(selector) -> dict[str, str | int]:
    # title
    title = selector.css(".title .ytd-video-primary-info-renderer::text").get()

    if title == None:
        title = selector.css(
            "yt-formatted-string.style-scope.ytd-watch-metadata span.style-scope.yt-formatted-string::text"
        ).get()

    print(title)

    # date
    date = selector.css("#info-strings yt-formatted-string::text").get()

    # views
    views = selector.css("span.bold.style-scope.yt-formatted-string::text").get()

    # likes
    likes = selector.css(
        "#segmented-like-button > ytd-toggle-button-renderer > yt-button-shape > button::attr(aria-label)"
    ).get()

    # comments
    try:
        comments_amount = int(
            selector.css("#count .count-text span:nth-child(1)::text")
            .get()
            .replace(",", "")
        )
    except Exception as e:
        comments_amount = 0

    # keywords
    # all_script_tags = selector.css("script").getall()
    # keywords = (
    #     "".join(
    #         re.findall(r'"keywords":\[(.*)\],"channelId":".*"', str(all_script_tags))
    #     )
    #     .replace('"', "")
    #     .split(",")
    # )

    print(title, views, likes, date, comments_amount)

    try:
        likes = re.search(r"(\d+)", likes).group()
    except Exception as e:
        likes = 0
    # views = re.search(r"(\d+)", views).group()

    return pd.DataFrame.from_dict(
        {
            "title": [title],
            "views": [views],
            "likes": [likes],
            "date": [date],
            "comments_amount": [comments_amount],
            # "keywords": [keywords],
        }
    )


def parse_channel(selector) -> pd.DataFrame:
    subscribers = selector.css(
        "#subscriber-count.style-scope.ytd-c4-tabbed-header-renderer::text"
    ).get()

    # subscribers = re.search(r"(\d+)", subscribers).group()

    # get recent 5 videos
    channel_name = (
        selector.css("div#channel-header-container yt-formatted-string#text::text")
        .get()
        .strip()
    )

    videos = selector.css(
        "div.style-scope.ytd-rich-item-renderer ytd-rich-grid-media div.style-scope.ytd-rich-grid-media a#thumbnail.yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail::attr(href)"
    ).getall()[:5]

    u, ind = np.unique(videos, return_index=True)
    videos = u[np.argsort(ind)]

    df = pd.DataFrame()
    for v in videos:
        print("Video: ", v)

        result = scroll_page(f"https://www.youtube.com{v}")

        info = get_info_from_video(result)
        info["channel_name"] = channel_name
        info["subscribers"] = subscribers
        # print(info)

        df = pd.concat([df, info])
        # df = df.append(info, ignore_index=True)

    print("Videos i got: ", len(videos))
    return df


def parse_top_channel(channel_link: str) -> pd.DataFrame:
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 3)
    driver.implicitly_wait(10)

    driver.get(channel_link)
    driver.maximize_window()

    # //*[@id="chips"]/yt-chip-cloud-chip-renderer[2]
    # wait for page to load

    try:
        wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="chips"]/yt-chip-cloud-chip-renderer[2]')
            )
        ).click()
    except Exception as e:
        print("Nao possui aba de videos em alta")
        # return empty dataframe
        # return pd.DataFrame()

    time.sleep(3)
    selector = Selector(text=driver.page_source)
    driver.quit()

    channel_name = (
        selector.css("div#channel-header-container yt-formatted-string#text::text")
        .get()
        .strip()
    )

    videos = selector.css(
        "div.style-scope.ytd-rich-item-renderer ytd-rich-grid-media div.style-scope.ytd-rich-grid-media a#thumbnail.yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail::attr(href)"
    ).getall()[:5]

    u, ind = np.unique(videos, return_index=True)
    videos = u[np.argsort(ind)]

    df = pd.DataFrame()
    for v in videos:
        # print("Video: ", v)

        result = scroll_page(f"https://www.youtube.com{v}")

        info = get_info_from_video(result)
        print(info)
        info["channel_name"] = channel_name
        # print(info)

        df = pd.concat([df, info])
        # df = df.append(info, ignore_index=True)

    print("Videos i got: ", len(videos))
    return df



Pegar dados

In [3]:

# %%
with open("canais.txt", "r") as f:
    # read each line as a item in a list and remove the \n
    canais = [line.strip() for line in f.readlines()]

base_path = f"./data/{canais[0].split('@')[-1]}/"

df = pd.DataFrame()
df_top = pd.DataFrame()
df_lifetime = pd.DataFrame()
for ch in canais:
    # recent videos
    result = scroll_page(ch + "/videos")
    df_channel = parse_channel(result)
    df = pd.concat([df, df_channel])

    # top videos
    df_top_channel = parse_top_channel(ch + "/videos")
    df_top = pd.concat([df_top, df_top_channel])

    # lifetime info
    df_lifetime_channel = get_info_from_channel(ch + "/about")
    df_lifetime = pd.concat([df_lifetime, df_lifetime_channel])


os.makedirs(base_path, exist_ok=True)

df.to_csv(base_path + "videos.csv", index=False)
df_top.to_csv(base_path + "top_videos.csv", index=False)
df_lifetime.to_csv(base_path + "lifetime.csv", index=False)


Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


Video:  /watch?v=875mNpeAZlg


ESPECIAL DE MAIO CENER MEGA-02
ESPECIAL DE MAIO CENER MEGA-02 202 visualizações Marque este vídeo como "Gostei" com mais 0 pessoa Estreou em 11 de mai. de 2023 0


Video:  /watch?v=vM0FsI2eYzo


ESPECIAL DE MAIO CENTER MEGA- 01
ESPECIAL DE MAIO CENTER MEGA- 01 176 visualizações Marque este vídeo como "Gostei" com mais 1 pessoa Estreou em 11 de mai. de 2023 0
Video:  /watch?v=8OK-jAkEu2E


...

Conheça a Casa São Pedro do Ariston!
Conheça a Casa São Pedro do Ariston! 131 mil visualizações Marque este vídeo como "Gostei" com mais 41 pessoas 27 de set. de 2021 3
                                  title                  views likes  \
0  Conheça a Casa São Pedro do Ariston!  131 mil visualizações    41   

                 date  comments_amount  
0  27 de set. de 2021                3  


Casa São Pedro do Centro de Osasco: Mais de 64 anos de tradição!
Casa São Pedro do Centro de Osasco: Mais de 64 anos de tradição! 130 mil visualizações Marque este vídeo como "Gostei" com mais 91 pessoas 6 de out. de 2021 5
                                               title                  views  \
0  Casa São Pedro do Centro de Osasco: Mais de 64...  130 mil visualizações   

  likes               date  comments_amount  
0    91  6 de out. de 2021                5  


A Super Casa São Pedro Cajamar está te esperando!
A Super Casa São Pedro Cajamar está te esperando! 124 mil visualizações Marque este vídeo como "Gostei" com mais 14 pessoas 17 de jan. de 2022 0
                                               title                  views  \
0  A Super Casa São Pedro Cajamar está te esperando!  124 mil visualizações   

  likes                date  comments_amount  
0    14  17 de jan. de 2022                0  


Um passeio pela Casa São Pedro da Inocêncio Seráfico!
Um passeio pela Casa São Pedro da Inocêncio Seráfico! 122 mil visualizações Marque este vídeo como "Gostei" com mais 34 pessoas 17 de jan. de 2022 2
                                               title                  views  \
0  Um passeio pela Casa São Pedro da Inocêncio Se...  122 mil visualizações   

  likes                date  comments_amount  
0    34  17 de jan. de 2022                2  
Videos i got:  5


{
  "channel_title": "Casa São Pedro",
  "date_joined": "22 de ago. de 2019",
  "total_views": "2.032.471 visualizações",
  "total_subs": "8,19 mil inscritos",
  "total_videos": "145"
}


last 30 days

In [ ]:

# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

print(canais)

base_url = "https://socialblade.com/youtube/channel/"

social_blade_data = []
for c in canais:
    response = scroll_page(base_url + c)

    # last 30 days views #socialblade-user-content > div:nth-child(23) > div:nth-child(3) > span
    views = response.css(
        "#socialblade-user-content > div:nth-child(23) > div:nth-child(3) > span::text"
    ).get()
    subs = response.css(
        "#socialblade-user-content > div:nth-child(23) > div:nth-child(2) > span::text"
    ).get()

    if views is None or subs is None:
        views = response.css(
            "#socialblade-user-content > div:nth-child(16) > div:nth-child(3) > span::text"
        ).get()
        subs = response.css(
            "#socialblade-user-content > div:nth-child(16) > div:nth-child(2) > span::text"
        ).get()

    social_blade_data.append({"Channel": c, "views": views, "subs": subs})

last_30days = pd.DataFrame(social_blade_data)
last_30days = last_30days.fillna("0")

last_30days.to_parquet(f'data/{canais[0].split("@")[-1]}/last_30_days.parquet', index=False)